# Doc2Vec Multiple models

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.utils import shuffle
import csv
import glob


%matplotlib inline

from scipy import stats, integrate
import matplotlib.pyplot as plt

%matplotlib inline

# Standard python helper libraries.
import collections
import itertools
import os
import re
import sys
import time


from scipy import stats
import scipy.optimize
from IPython.display import display, HTML

import nltk
#nltk.download()

# Helper libraries (see the corresponding py files in this notebook's directory).
import segment
import utils
import vocabulary
#import helpers


# Plotly imports.
import plotly.offline as plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go


import codecs
# This runs a shell command from the notebook.
!pip install plotly

import collections
from itertools import chain

import gensim
from gensim.models import Doc2Vec
from multiprocessing import cpu_count

##################################################################################################################
# Some basic stipping out to reduce vocabulary size
##################################################################################################################
# Strip out punctuations
import string
punctuations = list(string.punctuation)
punctuations.append("''")


# Function to stip out non-ASIC characters
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)


############################################################################################################################
# Actual code start
############################################################################################################################

path = "CSV/*.csv"
for file_name in glob.glob(path):
    df_full = pd.read_csv(file_name)
    print file_name
    df_full = df_full[df_full['claim_text_stemmed'].notnull()]
    df_full = df_full[df_full['appl_doc_number'].notnull()]    
    df_full = df_full.drop_duplicates(subset=[ 'claim_text_stemmed'], keep=False)
    # Get the number of sentences
    claims_range = len(df_full)
    df_all = df_full    

    # Get all the claims to create a corpus
    claims_array = []
    # Stip out items in the claims array
    for i in range(claims_range):
        claim = df_full.iloc[i]['claim_text_stemmed']
        # Remove non-ASCII characters
        claim=strip_non_ascii(claim)
        # Strip out punctuations
        claim = [i for i in nltk.word_tokenize(claim) if i not in punctuations]
        # Put them back as a sentence again
        claim = " ".join(claim)
        claims_array.append(claim)

    # We are going to examine the comments for some statistics
    # Get the words
    #tokens = list(utils.flatten(s.split() for s in claims_array))

    # "canonicalize_word" performs a few tweaks to the token stream of
    # the corpus.  For example, it replaces digits with DG allowing numbers
    # to aggregate together when we count them below.

    #token_feed = (utils.canonicalize_word(w) for w in tokens)
    #vocab = vocabulary.Vocabulary(token_feed)
    #print file_name, "Vocabulary size:", vocab.size

    # Get the array range 
    final_claims_range = len(claims_array)

    new_claims_array = claims_array
    new_claims_range = len(new_claims_array)

    


    # Doc2vec section
    doc2vec_words = []
    for f in range(new_claims_range):
        doc2vec_words.append(new_claims_array[f])
        doc2vec_words.append('\n')
    

    doc2vec_words = list(chain.from_iterable(doc2vec_words))
    doc2vec_words = ''.join(doc2vec_words)[:-1]

    # Build the sentences
    doc2vec_sentences = doc2vec_words.split('\n')
    


    cpus = cpu_count()

    def read_corpus():
        for i,sentence in enumerate(doc2vec_words.split('\n')):
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(sentence), [i])

    train_corpus = list(read_corpus())

    embedding_size = 500 # Dimension of the embedding vector.
    window = 8

    doc2vec_model = Doc2Vec(dm=1, dm_concat=1, size=embedding_size, window=window, 
                    negative=0,hs=0, min_count=1, workers=cpus, iter=10)
    doc2vec_model.build_vocab(train_corpus)

    #doc2vec_model.save('small_doc2vec_model')   
    #doc2vec_model = Doc2Vec.load('small_doc2vec_model')
    
    %%time
    doc2vec_model.train(train_corpus, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.iter)
    doc2vec_model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
    
    train = file_name.split("/")
    train = train[1].split(".")

    train_model = 'MODELS/'+train[0]+'_train'
    print "Writing", train_model
    doc2vec_model.save(train_model)    